# Processamento dos dados do ERA5

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d

In [2]:
def uv2id(u, v):
    """
    Conversao de u e v para intensidade e direcao
    """
    i = np.sqrt(u ** 2 + v ** 2)
    d = (np.rad2deg(np.arctan2(u, v))) % 360.
    return i, d

In [3]:
def read_era5_param(pathname, filename, datei, datef):
    """
    Leitura dos parâmetros do ERA5
    """
    ds1 = xr.open_dataset(pathname + filename)    
    ds = ds1.sel(longitude=-41.590557, latitude=-23.810278, method='nearest')
    ds = ds.sel(time=slice(datei, datef))
    df = ds.to_dataframe()
    df['ws'], df['wd'] = uv2id(df.u10, df.v10)
    return df

In [4]:
def read_era5_spec(pathname, filename, datei, datef):
    """
    Once decoded, the units of 2D wave spectra are m2 s radian-1
    """
    da = xr.open_dataarray(pathname + filename)
    da = da.sel(longitude=-41.5902, latitude=-23.8106, method='nearest')    
    da = da.sel(time=slice(datei, datef))    
    da = da.assign_coords(direction=np.arange(7.5, 352.5 + 15, 15))
    da = da.assign_coords(frequency=np.full(30, 0.03453) * (1.1 ** np.arange(0, 30)))
    da = 10 ** da
    da = da.fillna(0)

    # vetor de frequencia
    freq = pd.DataFrame(da.frequency.values)
    return da, freq

In [5]:
def create_era5_spec1d(da):
    """
    """
    # integra o espectro 2D em direcao para obter o espectro 1D
    a = [da.data[i,:,:].sum(axis=1) for i in range(da.data.shape[0])]

    # cria dataframe com o espectro 1D
    spec1d = pd.DataFrame(a, index=da.time.data)
    spec1d.index.name = 'date'
    return spec1d

In [6]:
def calc_era5_tp(spec1d):
    """
    """
    era5_tp = []
    for i in range(len(spec1d)):
        s = spec1d.iloc[i,:]
        era5_tp.append(1/freq.values[np.where(s == s.max())[0]][0])
    return np.array(era5_tp)

In [7]:
def plot_polar_spec(s2, radii, thetas, datet):
    """
    """
    fig = plt.figure(figsize=(11,11))

    ## Color-scale
    contour_levels = np.arange(0.00,s2.max(),0.1) # Manually set colorbar min/max values

    ax = plt.subplot(111, polar=True)
    ax.set_theta_direction(-1)
    ax.set_theta_zero_location("N")
    # ylabels = ([20,10,6.7,5,4])
    # ax.set_yticklabels(ylabels)

    colorax = ax.contour(thetas, radii, s2, contour_levels)


    ## Set titles and colorbar
    plt.suptitle('Polar Spectrum - ERA5 - ' + datet, fontsize=22, y=0.95, x=0.45)
    # title(startdate, fontsize=16, y=1.11)

    cbar = fig.colorbar(colorax)
    cbar.set_label('Energy Density (m*m/Hz/deg)', rotation=270, fontsize=16)
    cbar.ax.get_yaxis().labelpad = 30

    degrange = range(0,360,30)
    lines, labels = plt.thetagrids(degrange, labels=None)
    return fig

In [8]:
    datei = '2015-03-05 13:00:00'
    datef = '2015-04-07 06:00:00'

    path_data = os.environ['HOME'] + '/gdrive/coppe/doutorado/data/'
    path_out = os.environ['HOME'] + '/gdrive/coppe/doutorado/output/'

    # leitura dos dados
    param = read_era5_param(path_data, 'ERA5_windwave_param_CF01.nc', datei, datef)
    spec2, freq = read_era5_spec(path_data, 'ERA5_sfc_wspec_CF01.nc', datei, datef)

    # calculo do espectro 1D do ERA5
    spec1 = create_era5_spec1d(spec2)

    # calcula periodo de pico do ER5
    era5_tp = calc_era5_tp(spec1)

    # coloca a variavel de tp no parametro do ERA5
    param['tp'] = era5_tp

    param.index.name = 'date'

    # salva dados do ERA5
    param.to_csv(path_out + 'era5_windwave_param.csv')
    freq.to_csv(path_out + 'era5_wave_freq.csv')
    spec1.to_csv(path_out + 'era5_wave_spec1.csv')
    spec2.to_netcdf(path=path_out + 'era5_wave_spec2.nc')

## Plotagem polar do espectro 2D

In [9]:
# f = freq.values[:,0]
# d = np.deg2rad(spec2.direction.values)
# dd = np.concatenate(([0],d,[2*np.pi]))
# for i in range(len(spec2.time)):
#     s2 = spec2.values[i,:,:]
#     datet = str(spec2.time.values[i])[:-16]
#     ff = interp2d(d, f, s2, kind='cubic')
#     s22 = ff(dd, f)
#     s22[np.where(s22 < 0)] = 0
#     fig = plot_polar_spec(s2=s22, radii=f, thetas=dd, datet=datet)
#     fig.savefig(os.environ['HOME'] + '/Documents/doutorado/figs/spec2_polar_era5_{}.png'.format(
#         datet), bbox_inches='tight')
#     plt.close('all')
#     plt.show()